## Setup Colab

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
wdir = '/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
np.set_printoptions(linewidth=np.inf)

In [4]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


## Video: grid velocity plot

In [5]:
video_name = 'grid_velocity'

data_folder_name = 'data_extracted'
videos_dir = 'videos/' + video_name
n_frames = 29

In [6]:
df = pd.read_csv(os.path.join(wdir, data_folder_name, 'grid_data.csv'))

In [7]:
for i in range(0, n_frames):
  number_str = str(i).zfill(3)

  img_name =  'frame' + number_str + '.png'

  df_filtered = df[df['t'] == i]
  df_filtered = df_filtered.reset_index(drop=True)

  X = df_filtered['x'].to_numpy()
  Y = df_filtered['y'].to_numpy()
  VX = df_filtered['vx'].to_numpy()
  VY = df_filtered['vy'].to_numpy()

  fig, ax = plt.subplots(1, 1, figsize=(6,6))
  fig.suptitle('Frame ' + number_str)
  ax.quiver(X, Y, VX, VY, edgecolors='k')
  plt.gca().invert_yaxis()
  fig.savefig(os.path.join(wdir, videos_dir, img_name))
  plt.close(fig)

In [8]:
%cd {os.path.join(wdir, videos_dir)}
!ffmpeg -r 6 -i frame%03d.png -vcodec mpeg4 -y {video_name}.mp4

/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a/videos/grid_velocity
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --en

## Video: grid speed plot

In [9]:
video_name = 'grid_speed'

data_folder_name = 'data_extracted'
videos_dir = 'videos/' + video_name
n_frames = 29

In [10]:
df = pd.read_csv(os.path.join(wdir, data_folder_name, 'grid_data.csv'))

In [11]:
def get_matrix(X,Y,U):

  def find_xy_ind(xval, yval, xarray, yarray):    
    for index in range(0, len(xarray)):
      if abs(xarray[index] - xval) < 0.0001 and abs(yarray[index] - yval) < 0.0001:
        return index

  X_unique=np.unique(X)
  Y_unique=np.unique(Y)

  matriz_final = np.empty([X_unique.shape[0], Y_unique.shape[0]])

  for i in range(X_unique.shape[0]):
    for j in range(Y_unique.shape[0]):
        xval = X_unique[i]
        yval = Y_unique[j]
        index = find_xy_ind(xval, yval, X, Y)
        matriz_final[i,j] = U[index]
  
  matriz_final = matriz_final.transpose()
  #matriz_final = np.flip(matriz_final, axis = 0)

  return matriz_final

In [12]:
X = [-200,-200,-200,-200,-200, -150, -150, -150,-150, -150, -100, -100, -100, -100, -100]
Y = [-60, -120, 0, 30, -30, -60, -120, 0, 30, -30, -60, -120, 0, 30, -30]
Z = [0.2, 0.1, 0, -0.32, 0.4, -0.2, 0.5, -0.1, 0.3, 0.4, -0.7, 0.7, 0.6, -0.3, 0.9]
print(get_matrix(X,Y,Z))

[[ 0.1   0.5   0.7 ]
 [ 0.2  -0.2  -0.7 ]
 [ 0.4   0.4   0.9 ]
 [ 0.   -0.1   0.6 ]
 [-0.32  0.3  -0.3 ]]


In [13]:
for i in range(0, n_frames):
  number_str = str(i).zfill(3)

  img_name =  'frame' + number_str + '.png'

  df_filtered = df[df['t'] == i]
  df_filtered = df_filtered.reset_index(drop=True)

  X = df_filtered['x'].to_numpy()
  Y = df_filtered['y'].to_numpy()
  U = df_filtered['u'].to_numpy()

  A = get_matrix(X,Y,U)

  fig, ax = plt.subplots(1, 1, figsize=(6,6))
  fig.suptitle('Frame ' + number_str)
  x = ax.imshow(A, cmap='hot', interpolation='nearest', vmin = 0, vmax = 60, origin = 'upper')
  plt.colorbar(x)
  fig.savefig(os.path.join(wdir, videos_dir, img_name))
  plt.close(fig)

In [14]:
%cd {os.path.join(wdir, videos_dir)}
!ffmpeg -r 6 -i frame%03d.png -vcodec mpeg4 -y {video_name}.mp4

/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a/videos/grid_speed
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enabl

## Video: grid packing fraction plot

In [15]:
video_name = 'grid_packing'

data_folder_name = 'data_extracted'
videos_dir = 'videos/' + video_name
n_frames = 29

In [16]:
df = pd.read_csv(os.path.join(wdir, data_folder_name, 'grid_data.csv'))

In [17]:
def get_matrix(X,Y,U):

  def find_xy_ind(xval, yval, xarray, yarray):    
    for index in range(0, len(xarray)):
      if abs(xarray[index] - xval) < 0.0001 and abs(yarray[index] - yval) < 0.0001:
        return index

  X_unique=np.unique(X)
  Y_unique=np.unique(Y)

  matriz_final = np.empty([X_unique.shape[0], Y_unique.shape[0]])

  for i in range(X_unique.shape[0]):
    for j in range(Y_unique.shape[0]):
        xval = X_unique[i]
        yval = Y_unique[j]
        index = find_xy_ind(xval, yval, X, Y)
        matriz_final[i,j] = U[index]
  
  matriz_final = matriz_final.transpose()
  #matriz_final = np.flip(matriz_final, axis = 0)
  return matriz_final

In [18]:
for i in range(0, n_frames):
  number_str = str(i).zfill(3)

  img_name =  'frame' + number_str + '.png'

  df_filtered = df[df['t'] == i]
  df_filtered = df_filtered.reset_index(drop=True)

  X = df_filtered['x'].to_numpy()
  Y = df_filtered['y'].to_numpy()
  Phi = df_filtered['phi'].to_numpy()

  A = get_matrix(X,Y,Phi)

  fig, ax = plt.subplots(1, 1, figsize=(6,6))
  fig.suptitle('Frame ' + number_str)
  x = ax.imshow(A, cmap='viridis', interpolation='nearest', vmin = 0.2, vmax = 0.7, origin = 'upper')
  plt.colorbar(x)
  fig.savefig(os.path.join(wdir, videos_dir, img_name))
  plt.close(fig)

In [19]:
%cd {os.path.join(wdir, videos_dir)}
!ffmpeg -r 6 -i frame%03d.png -vcodec mpeg4 -y {video_name}.mp4

/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a/videos/grid_packing
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --ena

## Video: packing fraction and number density scatter plot

In [20]:
video_name = 'packing_number_scatter'

data_folder_name = 'data_extracted'
videos_dir = 'videos/' + video_name
n_frames = 29

In [21]:
df = pd.read_csv(os.path.join(wdir, data_folder_name, 'grid_data.csv'))

In [22]:
for i in range(0, n_frames):
  number_str = str(i).zfill(3)

  img_name =  'frame' + number_str + '.png'

  df_filtered = df[df['t'] == i]
  df_filtered = df_filtered.reset_index(drop=True)

  Phi = df_filtered['phi'].to_numpy()
  N = df_filtered['N'].to_numpy()

  fig, ax = plt.subplots(1, 1, figsize=(6,6))
  fig.suptitle('Frame ' + number_str)
  ax.scatter(Phi, N, 2, 'green')
  ax.set_xlabel('Packing fraction')
  ax.set_ylabel('Number density')
  ax.set_xlim((0.2,0.75))
  ax.set_ylim((5,25))
  fig.savefig(os.path.join(wdir, videos_dir, img_name))
  plt.close(fig)

In [23]:
%cd {os.path.join(wdir, videos_dir)}
!ffmpeg -r 6 -i frame%03d.png -vcodec mpeg4 -y {video_name}.mp4

/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a/videos/packing_number_scatter
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libs

## Video: packing fraction and speed scatter plot

In [24]:
video_name = 'packing_speed_scatter'

data_folder_name = 'data_extracted'
videos_dir = 'videos/' + video_name
n_frames = 29

In [25]:
df = pd.read_csv(os.path.join(wdir, data_folder_name, 'grid_data.csv'))

In [26]:
for i in range(0, n_frames):
  number_str = str(i).zfill(3)

  img_name =  'frame' + number_str + '.png'

  df_filtered = df[df['t'] == i]
  df_filtered = df_filtered.reset_index(drop=True)

  Phi = df_filtered['phi'].to_numpy()
  U = df_filtered['u'].to_numpy()
  Phi_inv = 1/Phi

  fig, ax = plt.subplots(1, 1, figsize=(6,6))
  fig.suptitle('Frame ' + number_str)
  ax.scatter(Phi_inv, U, 2, 'green')
  ax.set_xlabel('Inverse of packing fraction')
  ax.set_ylabel(r'Speed ($\mu$m/h)')
  ax.set_xlim((1.4,5))
  ax.set_ylim((4,65))
  fig.savefig(os.path.join(wdir, videos_dir, img_name))
  plt.close(fig)

In [27]:
%cd {os.path.join(wdir, videos_dir)}
!ffmpeg -r 6 -i frame%03d.png -vcodec mpeg4 -y {video_name}.mp4

/content/gdrive/Othercomputers/My MacBook Air/cell-jamming/experiment_a/videos/packing_speed_scatter
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsp